# The Movie Database (Tmdb API)
## Gather movie-infos from Tmdb database 

Rate Limit: 40 items per 10 seconds <br> 
(Not so) proudly built with ChatGPT 


In [1]:
from themoviedb import TMDb
import requests
import pandas as pd
import time
import numpy as np

In [2]:
# TMDb API-Key (ersetzen!)
API_KEY = "1e99de5ed0cf038c6f479de3de969bdf"
BASE_URL = "https://api.themoviedb.org/3/movie/"

# CSV mit TMDb-IDs einlesen
input_file = "../input/tmdb_api/links_1m.csv"  # Datei mit einer Spalte "tmdb_id"
output_file = "tmdb_filmdaten.csv"

In [14]:
movies = pd.read_parquet("../movies.parquet", engine='fastparquet')
movies = movies.replace("", np.nan)

## TMDB API Abfrage 
Skript #1

In [10]:
df = pd.read_csv(input_file)

def fetch_movie_data(movie_id):
    """ Holt Filmdetails und Reviews von TMDb """
    try:
        # Filmdetails abrufen
        details_url = f"{BASE_URL}{movie_id}?api_key={API_KEY}&language=en"
        details = requests.get(details_url).json()
        
        # Basisdaten
        release_year = details.get("release_date", "")[:4]  # Nur Jahr
        overview = details.get("overview", "Keine Beschreibung verfügbar")

        # Reviews abrufen
        # reviews_url = f"{BASE_URL}{movie_id}/reviews?api_key={API_KEY}&language=de&page=1"
        # reviews = requests.get(reviews_url).json()

        # review_count = reviews.get("total_results", 0)
        # review_texts = [r["content"] for r in reviews.get("results", [])[:3]]  # Max. 3 Reviews
        
        return {
            "tmdb_id": movie_id,
            "release_year": release_year,
            "overview": overview,
            # "review_count": review_count,
            # "reviews": " | ".join(review_texts)  # Reviews als String zusammenfügen
        }
    except Exception as e:
        print(f"Fehler bei TMDb-ID {movie_id}: {e}")
        return None

# Daten abrufen in Batches von 40
batch_size = 40
movie_data = []

for i in range(0, len(df), batch_size):
    batch = df["tmdbId"][i:i+batch_size]  # 40 IDs holen
    print(f"Verarbeite IDs {i+1} bis {i+len(batch)}...")
    
    batch_results = [fetch_movie_data(mid) for mid in batch]
    movie_data.extend(batch_results)
    
    # Zwischenspeichern nach jedem Batch
    pd.DataFrame(movie_data).to_csv(output_file, index=False, encoding="utf-8")
    print(f"Zwischenspeicherung nach {len(movie_data)} Filmen.")

    # Warten nach jedem Batch, außer beim letzten
    if i + batch_size < len(df):
        print("Warte 10 Sekunden, um das Rate Limit einzuhalten...")
        time.sleep(10)

print(f"Ergebnisse gespeichert in {output_file}")


Verarbeite IDs 1 bis 40...
Zwischenspeicherung nach 40 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 41 bis 80...
Zwischenspeicherung nach 80 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 81 bis 120...
Zwischenspeicherung nach 120 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 121 bis 160...
Zwischenspeicherung nach 160 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 161 bis 200...
Zwischenspeicherung nach 200 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 201 bis 240...
Zwischenspeicherung nach 240 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 241 bis 280...
Zwischenspeicherung nach 280 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 281 bis 320...
Zwischenspeicherung nach 320 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 321 bis 360...


KeyboardInterrupt: 

## TMDB API Abfrage 
Skript #2

In [2]:
import requests
import pandas as pd
import time
from datetime import datetime

# Datum und Uhrzeit für Dateinamen
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")  # Format: YYYY-MM-DD_HH-MM-SS

# TMDb API-Key (ersetzen!)
API_KEY = "1e99de5ed0cf038c6f479de3de969bdf"
BASE_URL = "https://api.themoviedb.org/3/movie/"

# CSV mit TMDb-IDs einlesen
input_file = "../input/tmdb_api/links_1m.csv"  # Datei mit einer Spalte "tmdb_id"
output_file = f"tmdb_filmdaten_{timestamp}.csv"

df = pd.read_csv(input_file)

def fetch_movie_data(movie_id):
    """ Holt alle relevanten Filmdaten von TMDb inklusive Cast, Crew & Director """
    try:
        # Filmdetails abrufen mit Credits (Cast & Crew) und Keywords
        details_url = f"{BASE_URL}{movie_id}?api_key={API_KEY}&language=en&append_to_response=keywords,credits"
        details = requests.get(details_url).json()

        # Cast & Crew-Daten extrahieren
        cast_list = details.get("credits", {}).get("cast", [])
        crew_list = details.get("credits", {}).get("crew", [])

        # Hauptdarsteller (Top 5)
        top_cast = ", ".join([actor["name"] for actor in cast_list[:5]])

        # Regisseure aus der Crew extrahieren
        directors = ", ".join([member["name"] for member in crew_list if member["job"] == "Director"])

        # Optional: gesamte Crew (erste 10 Mitglieder)
        crew_members = ", ".join([member["name"] for member in crew_list[:10]])

        return {
            "movie_id": movie_id,
            "budget": details.get("budget", 0),
            "genres": ", ".join([g["name"] for g in details.get("genres", [])]),
            "homepage": details.get("homepage", ""),
            "keywords": ", ".join([k["name"] for k in details.get("keywords", {}).get("keywords", [])]),
            "original_language": details.get("original_language", ""),
            "original_title": details.get("original_title", ""),
            "overview": details.get("overview", "Keine Beschreibung verfügbar"),
            "popularity": details.get("popularity", 0.0),
            "production_companies": ", ".join([c["name"] for c in details.get("production_companies", [])]),
            "production_countries": ", ".join([c["name"] for c in details.get("production_countries", [])]),
            "release_date": details.get("release_date", ""),
            "revenue": details.get("revenue", 0),
            "runtime": details.get("runtime", 0.0),
            "spoken_languages": ", ".join([l["name"] for l in details.get("spoken_languages", [])]),
            "status": details.get("status", ""),
            "tagline": details.get("tagline", ""),
            "title": details.get("title", ""),
            "vote_average": details.get("vote_average", 0.0),
            "vote_count": details.get("vote_count", 0),
            "cast": top_cast,  # Hauptdarsteller (Top 5)
            "crew": crew_members,  # Erste 10 Crew-Mitglieder
            "director": directors  # Nur die Regisseure
        }
    except Exception as e:
        print(f"Fehler bei TMDb-ID {movie_id}: {e}")
        return None

# Daten abrufen in Batches von 40 (wegen Rate Limit)
batch_size = 40
movie_data = []

for i in range(0, len(df), batch_size):
    batch = df["id"][i:i+batch_size]  # 40 IDs holen
    print(f"Verarbeite IDs {i+1} bis {i+len(batch)}...")

    batch_results = [fetch_movie_data(mid) for mid in batch]
    movie_data.extend(batch_results)
    
    # Zwischenspeicherung nach jedem Batch
    pd.DataFrame(movie_data).to_csv(output_file, index=False, encoding="utf-8")
    print(f"Zwischenspeicherung nach {len(movie_data)} Filmen.")

    # Warten nach jedem Batch, außer beim letzten
    if i + batch_size < len(df):
        print("Warte 10 Sekunden, um das Rate Limit einzuhalten...")
        time.sleep(10)

print(f"Ergebnisse gespeichert in {output_file}")

Verarbeite IDs 1 bis 40...
Zwischenspeicherung nach 40 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 41 bis 80...
Zwischenspeicherung nach 80 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 81 bis 120...
Zwischenspeicherung nach 120 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 121 bis 160...
Zwischenspeicherung nach 160 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 161 bis 200...
Zwischenspeicherung nach 200 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 201 bis 240...
Zwischenspeicherung nach 240 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 241 bis 280...
Zwischenspeicherung nach 280 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 281 bis 320...
Zwischenspeicherung nach 320 Filmen.
Warte 10 Sekunden, um das Rate Limit einzuhalten...
Verarbeite IDs 321 bis 360...
Zwischenspeicherung nach 3

KeyboardInterrupt: 

## TMDB API Abfrage 
Skript #3 - verbesserte parallelisierung und Anpassung an neue rat Limits

In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime

# Datum und Uhrzeit für Dateinamen
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")  # Format: YYYY-MM-DD_HH-MM-SS

# TMDb API-Key (ersetzen!)
API_KEY = "1e99de5ed0cf038c6f479de3de969bdf"
BASE_URL = "https://api.themoviedb.org/3/movie/"

# CSV mit TMDb-IDs einlesen
input_file = "../input/tmdb_api/links_1m.csv"  # Datei mit einer Spalte "tmdb_id"
output_file = f"tmdb_filmdaten_{timestamp}.csv"

df = pd.read_csv(input_file)

# Request-Session für bessere Performance
session = requests.Session()

def fetch_movie_data(movie_id):
    """ Holt alle relevanten Filmdaten von TMDb inklusive Cast, Crew & Director """
    try:
        # Filmdetails abrufen mit Credits (Cast & Crew) und Keywords
        details_url = f"{BASE_URL}{movie_id}?api_key={API_KEY}&language=en&append_to_response=keywords,credits"
        response = session.get(details_url)
        
        # Falls die API das Limit erreicht, warten & erneut versuchen
        if response.status_code == 429:
            print(f"Rate Limit erreicht für {movie_id}, warte 5 Sekunden...")
            time.sleep(5)
            return fetch_movie_data(movie_id)  # Erneuter Versuch
        
        details = response.json()

        # Cast & Crew-Daten extrahieren
        cast_list = details.get("credits", {}).get("cast", [])
        crew_list = details.get("credits", {}).get("crew", [])

        # Hauptdarsteller (Top 5)
        top_cast = ", ".join([actor["name"] for actor in cast_list[:5]])

        # Regisseure aus der Crew extrahieren
        directors = ", ".join([member["name"] for member in crew_list if member["job"] == "Director"])

        return {
            "movie_id": movie_id,
            "budget": details.get("budget", 0),
            "genres": ", ".join([g["name"] for g in details.get("genres", [])]),
            "homepage": details.get("homepage", ""),
            "keywords": ", ".join([k["name"] for k in details.get("keywords", {}).get("keywords", [])]),
            "original_language": details.get("original_language", ""),
            "original_title": details.get("original_title", ""),
            "overview": details.get("overview", "Keine Beschreibung verfügbar"),
            "popularity": details.get("popularity", 0.0),
            "production_companies": ", ".join([c["name"] for c in details.get("production_companies", [])]),
            "production_countries": ", ".join([c["name"] for c in details.get("production_countries", [])]),
            "release_date": details.get("release_date", ""),
            "revenue": details.get("revenue", 0),
            "runtime": details.get("runtime", 0.0),
            "spoken_languages": ", ".join([l["name"] for l in details.get("spoken_languages", [])]),
            "status": details.get("status", ""),
            "tagline": details.get("tagline", ""),
            "title": details.get("title", ""),
            "vote_average": details.get("vote_average", 0.0),
            "vote_count": details.get("vote_count", 0),
            "cast": top_cast,
            "director": directors,
            "poster_url": f"https://image.tmdb.org/t/p/w500{details.get('poster_path', '')}" if details.get("poster_path") else "",
        }
    except Exception as e:
        print(f"Fehler bei TMDb-ID {movie_id}: {e}")
        return None

# Limitierung auf 50 Requests pro Sekunde
batch_size = 50
movie_data = []
start_time = time.time()

for i in range(0, len(df), batch_size):
    batch = df["id"][i:i+batch_size]  # 50 IDs holen
    print(f"Verarbeite IDs {i+1} bis {i+len(batch)}...")

    batch_results = [fetch_movie_data(mid) for mid in batch]
    movie_data.extend(batch_results)

    # Zwischenspeicherung nach jedem Batch
    pd.DataFrame(movie_data).to_csv(output_file, index=False, encoding="utf-8")
    print(f"Zwischenspeicherung nach {len(movie_data)} Filmen.")

    # Rate-Limit-Kontrolle
    elapsed_time = time.time() - start_time
    if elapsed_time < 1:
        sleep_time = 1 - elapsed_time
        print(f"Warte {sleep_time:.2f} Sekunden für Rate-Limit...")
        time.sleep(sleep_time)
    
    start_time = time.time()

print(f"Ergebnisse gespeichert in {output_file}")


Verarbeite IDs 1 bis 50...
Zwischenspeicherung nach 50 Filmen.
Warte 0.04 Sekunden für Rate-Limit...
Verarbeite IDs 51 bis 100...
Zwischenspeicherung nach 100 Filmen.
Warte 0.22 Sekunden für Rate-Limit...
Verarbeite IDs 101 bis 150...
Zwischenspeicherung nach 150 Filmen.
Warte 0.39 Sekunden für Rate-Limit...
Verarbeite IDs 151 bis 200...
Zwischenspeicherung nach 200 Filmen.
Verarbeite IDs 201 bis 250...
Zwischenspeicherung nach 250 Filmen.
Verarbeite IDs 251 bis 300...
Zwischenspeicherung nach 300 Filmen.
Verarbeite IDs 301 bis 350...
Zwischenspeicherung nach 350 Filmen.
Verarbeite IDs 351 bis 400...
Zwischenspeicherung nach 400 Filmen.
Verarbeite IDs 401 bis 450...
Zwischenspeicherung nach 450 Filmen.
Verarbeite IDs 451 bis 500...
Zwischenspeicherung nach 500 Filmen.
Verarbeite IDs 501 bis 550...
Zwischenspeicherung nach 550 Filmen.
Verarbeite IDs 551 bis 600...
Zwischenspeicherung nach 600 Filmen.
Verarbeite IDs 601 bis 650...
Zwischenspeicherung nach 650 Filmen.
Verarbeite IDs 651 b